In [18]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [6]:
def lp_dist(p:float):
    # _lp_dist has access to its outer function's scope.
    def _lp_dist(x1:np.ndarray, x2:np.ndarray) -> np.ndarray:
        """Get the pairwise Lp-distances between points.
        
        Args:
            x1: an m by d matrix where each row corresponds to a point to
                be compared with each point in x2.    
            x2: an n by d matrix where each row corresponds to a point to
                be compared with each point in x1. 
            p: p > 0

        Returns:
            a distance matrix of dimensions m by n where the (i,j)-th entry
            corresponds to the distance
            between points x1[i, :] and x2[j, :]
        """
        m = x1.shape[0]
        n = x2.shape[0]
        d = x1.shape[1]

        # Compare each row of x1 to each row of x2.
        # To do this, make views x1_view and x2_view
        # into x1 and x2, respectively.  x1_view and x2_view have the same
        # entries as x1 and x2, but they are repeated in a nice
        # configuration. 
        # Example:
        # x1 = np.array([
        # 	[1, 0, 0],
        # 	[0.45, 1.24, -0.02]])
        # x2 = np.array([
        # 	[0.9, -0.1, 0.3],
        # 	[0.8, -0.6, 0.4],
        # 	[0.2, -1.4, -0.4]])
        # x1_view
        # array([[[ 1.  ,  0.  ,  0.  ],
        #     [ 0.45,  1.24, -0.02]],
        #
        #    [[ 1.  ,  0.  ,  0.  ],
        #     [ 0.45,  1.24, -0.02]],
        #
        #    [[ 1.  ,  0.  ,  0.  ],
        #     [ 0.45,  1.24, -0.02]]])
        # x2_view
        # array([[[ 0.9, -0.1,  0.3],
        #         [ 0.8, -0.6,  0.4],
        #         [ 0.2, -1.4, -0.4]],
        #
        #        [[ 0.9, -0.1,  0.3],
        #         [ 0.8, -0.6,  0.4],
        #         [ 0.2, -1.4, -0.4]]])
        
        x1_view = np.broadcast_to(
            array=x1, 
            shape=(n, m, d)
        )
        
        x2_view = np.broadcast_to(
            array=x2, 
            shape=(m, n, d)
        )

        # Make sure our views have the same dimensions.
        x2_view_2 = np.swapaxes(x2_view, axis1=0, axis2=1)
        # Example:
        # x2_view_2
        # array([[[ 0.9, -0.1,  0.3],
        #         [ 0.9, -0.1,  0.3]],
        #
        #        [[ 0.8, -0.6,  0.4],
        #         [ 0.8, -0.6,  0.4]],
        #
        #        [[ 0.2, -1.4, -0.4],
        #         [ 0.2, -1.4, -0.4]]])
        
        # Now, x1_view and x2_view_2 are broadcast-able.

        return np.power(
            np.sum(
                np.power(
                    np.abs(x1_view - x2_view_2), 
                    p
                ), 
                axis=2
            ).T, 
            1.0/p
        )
    
    return _lp_dist

In [7]:
euclidean_dist = lp_dist(p=2)

In [27]:
def oklch_to_oklab(oklch:np.ndarray) -> np.ndarray:
    """Convert OKLCH color coordinates to OKLAB.
    
    Args:
        oklch: an n by 3 matrix where each
            row is an OKLCH color.
    """
    h = np.radians(oklch[:, 2])
    c = oklch[:, 1]
    a = c * np.cos(h)
    b = c * np.sin(h)

    oklab_mat = np.empty(shape=(oklch.shape[0], oklch.shape[1]))

    oklab_mat[:, 0] = oklch[:, 0]
    oklab_mat[:, 1] = a
    oklab_mat[:, 2] = b

    return oklab_mat

def oklab_to_linear_srgb(oklab: np.ndarray):
    """Convert OKLAB color coordinates to Linear SRGB.
    
    See: https://bottosson.github.io/posts/oklab/
    Args:
        oklab: an n by 3 matrix where each
            row is an OKLAB color.
    """ 
    constants_1 = np.array([ 
        [1.        ,  1.        ,  1.        ],
        [ 0.39633778, -0.10556135, -0.08948418],
        [0.21580376, -0.06385417, -1.29148555]
    ])

    lms_mat_1 = oklab @ constants_1

    lms_mat_2 = lms_mat_1 * lms_mat_1 * lms_mat_1

    constants_2 = np.array([
        [4.0767416621, -1.2684380046, -0.0041960863],
        [-3.3077115913, 2.6097574011, -0.7034186147],
        [0.2309699292, -0.3413193965, 1.7076147010]
    ])

    return lms_mat_2 @ constants_2

def oklch_to_linear_srgb(oklch):
    oklab = oklch_to_oklab(oklch)
    srgb_lin = oklab_to_linear_srgb(oklab)
    return srgb_lin

def linear_srgb_to_srgb(srgb_lin):
    """
    https://www.image-engineering.de/library/technotes/958-how-to-convert-between-srgb-and-ciexyz
    """
    part_1 = np.where(
        srgb_lin <= 0.0031308,
        srgb_lin * 12.92,
        1.055 * srgb_lin ** (1.0/2.4) - 0.055
    )

    part_2 = 255 * part_1
    return part_2

def oklab_to_srgb(oklab):
    srgb_lin = oklab_to_linear_srgb(oklab)
    srgb = linear_srgb_to_srgb(srgb_lin)
    return srgb

def oklch_to_srgb(oklch):
    oklab = oklch_to_oklab(oklch)
    srgb_lin = oklab_to_linear_srgb(oklab)
    srgb = linear_srgb_to_srgb(srgb_lin)
    return srgb

def rgb_mat_to_str(rgb_mat: np.ndarray) -> list[str]:
    n_colors = rgb_mat.shape[0]
    rgb_mat_list = []
    for i in range(n_colors):
        rgb_mat_str = f"rgb({','.join([str(u) for u in rgb_mat[i, :]])})" 
        rgb_mat_list.append(rgb_mat_str)

    return rgb_mat_list


In [9]:
oklch_mat = np.array([
    [0.67, 0.0038, 174.42],
    [0.38, 0.0515, 259.43],
    [0.77, 0.0142, 241.02],
    [0.52, 0.084, 41.62],
    [0.52, 0.084, 156.63]
])

In [10]:
oklab_mat = oklch_to_oklab(oklch_mat)
print(oklab_mat)

[[ 6.70000000e-01 -3.78199333e-03  3.69494877e-04]
 [ 3.80000000e-01 -9.44698808e-03 -5.06261239e-02]
 [ 7.70000000e-01 -6.87996093e-03 -1.24220022e-02]
 [ 5.20000000e-01  6.27955684e-02  5.57917250e-02]
 [ 5.20000000e-01 -7.71088455e-02  3.33200533e-02]]


In [11]:
print(euclidean_dist(oklab_mat, oklab_mat))

[[0.         0.29450407 0.10086238 0.17321719 0.17018392]
 [0.29450407 0.         0.39187516 0.19011508 0.17670622]
 [0.10086238 0.39187516 0.         0.26834268 0.26367486]
 [0.17321719 0.19011508 0.26834268 0.         0.14169764]
 [0.17018392 0.17670622 0.26367486 0.14169764 0.        ]]


In [12]:
oklch_to_srgb(oklch_mat)

array([[146.88492084, 149.90574936, 148.89872796],
       [ 49.81224621,  66.83995971,  93.87444553],
       [172.57916942, 181.57985709, 188.58161995],
       [145.65027774,  87.80764665,  65.614226  ],
       [ 59.26199398, 119.39472334,  84.87182088]])

In [14]:
def get_medioid(x:np.ndarray, d) -> np.ndarray:
    """Get the medioid using the distance function d.
    
    Args:
        x: an n by p matrix (numpy.ndarray) where each row corresponds to 
            a point in R^p.
        d: a function that takes two matrices (numpy.ndarray) as its first two
            positional arguments.  The function
            should return a matrix (numpy.ndarray) with entries that correspond
            to pairwise distances between rows of the input
            matrices.

    Returns:
        medioid point as a 1-dimensional array
    """
    medioid_index = np.argmin(np.sum(d(x, x), axis=0, keepdims=False))
    return x[medioid_index, :].ravel()

In [17]:
oklab_to_srgb(get_medioid(x=oklab_mat, d=euclidean_dist))

array([146.88492084, 149.90574936, 148.89872796])

In [29]:
srgb_mat = oklch_to_srgb(oklch_mat)

In [30]:
rgb_list = rgb_mat_to_str(srgb_mat)

x=oklab_mat[:, 0]
y=oklab_mat[:, 1]
z=oklab_mat[:, 2]
# https://stackoverflow.com/questions/70340331/how-can-i-manually-color-each-point-in-my-scatter-plot-in-plotly
fig = go.Figure(
    go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode="markers",
        marker_color=rgb_list,
    )
)

fig.show()